# Refinitiv Data Library for Python

## Content layer - IPA - Swap stream - With events
This notebook demonstrates how to use a Swap pricing analytics stream either directly from the Refinitiv Data Platform or via Refinitiv Workspace or CodeBook.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

## Some Imports to start with

In [1]:
import refinitiv.data as rd
from refinitiv.data.content.ipa.financial_contracts import swap
import datetime

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
rd.open_session()

<refinitiv.data.session._desktop_session.Definition object at 0x7fd27895ea90 {name='default'}>

## About events
When you use a QuantitativeDataStream object to open streaming instruments with events, you need to define callback functions that are called by a background execution thread when new events are received for the instruments you requested. There are 3 different types of functions you can use depending on the type of events you are interrested in. These callback functions are optionnal meaning that you do not have to use all of them. 

### The 3 event types and their related callback functions:
 - **Response events:** 
 
 Callback functions used to receive Response events take 3 parameters: 
  - The QuantitativeDataStream that received the event.
  - The data of the concerned instrument.
  - The field names of the data.
 
 
 - **Update events:** 
 
 Callback functions used to receive Update events take 3 parameters 
  - The QuantitativeDataStream that received the event.
  - The data of the concerned instrument.
  - The field names of the data.
 
 
 - **State events:** 
 
 Callback functions used to receive Update events take 3 parameters 
  - The QuantitativeDataStream that received the event.
  - The state of the stream.

## Retrieve data

### Define the Swap

In [3]:
swap = swap.Definition(
        tenor="3Y",
        instrument_code="EURAB6E3Y=",
        pricing_parameters=swap.PricingParameters(
            index_convexity_adjustment_integration_method=swap.IndexConvexityAdjustmentIntegrationMethod.RIEMANN_SUM,
            index_convexity_adjustment_method=swap.IndexConvexityAdjustmentMethod.BLACK_SCHOLES,
            valuation_date="2021-03-09",
        ),
        fields=[
            "InstrumentDescription",
            "ValuationDate",
            "StartDate",
            "FixedRate",
        ],
    )

### Get the stream and register callbacks to receive events

In [4]:
def display_event(eventType, data, headers):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    print(data)
    print(headers)
    
stream = (
        swap.get_stream()
        .on_response(lambda stream, data, headers: display_event("response", data, headers))
        .on_update(lambda stream, data, headers: display_event("update", data, headers))
        .on_state(lambda stream, data: display_event("state", data, None))
    )

### Open the stream

In [6]:
stream.open()

----------------------------------------------------------
>>> response event received at 17:12:50.332236
[["PaidLeg[Curr:'EUR' Type:'Fixed']", '2021-03-09T00:00:00Z', '2021-03-11T00:00:00Z', -0.00436499989985321], ["ReceivedLeg[Curr:'EUR' Type:'Float']", '2021-03-09T00:00:00Z', '2021-03-11T00:00:00Z', None]]
['InstrumentDescription', 'ValuationDate', 'StartDate', 'FixedRate']


<OpenState.Opened: 1>

### Close the stream

In [7]:
stream.close()

<OpenState.Closed: 3>

## Close the session

In [8]:
rd.close_session()